In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile 
from pathlib import Path
import os
from tqdm import tqdm
import sys
sys.path.append(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\10_code\UTvsXCT-preprocessing')
from preprocess_tools import aligner,onlypores,io,pore_filtering

In [2]:
#load the onlypores to filter

file_path = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Fabricacion Nacho\02_Laminados_Nacho\Probetas\Na_01\Na_01_bottom_2800x214x3942_reslice_top_onlypores.tif')

volume = io.load_tif(file_path)

# Size filtering

We filter pores that are smaller than 8 pixels.

A further research was done to determine this value.

Steps:

1. Label the pores to separate them as individual objects

2. Delete the ones smaller than 8 pixels

In [3]:
from skimage import measure,morphology

# Label the pores in the volume
labeled_pores = measure.label(volume, connectivity=1)

#filter
filtered_pores = morphology.remove_small_objects(labeled_pores, min_size=8)

# Flat filtering

We are going to filter the pores that are not 3D, i mean, a pore must have at least 2 pixels in each axes.

Javi told me something like this (but more technical).

In [4]:
from skimage.measure import regionprops

props = regionprops(filtered_pores)

# Create a mask to keep only the objects that are 2 pixels wide in each axis
mask = np.zeros_like(filtered_pores, dtype=bool)

for prop in tqdm(props):
    # Get the bounding box of the object
    min_row, min_col, min_depth, max_row, max_col, max_depth = prop.bbox
    # Check if the object is 2 pixels wide in each axis
    if (max_row - min_row >= 2) and (max_col - min_col >= 2) and (max_depth - min_depth >= 2):
        mask[prop.slice] = filtered_pores[prop.slice] == prop.label

# Apply the mask to the labeled volume
filtered_pores_2 = filtered_pores * mask

100%|██████████| 137206/137206 [00:02<00:00, 64597.39it/s]


# Full filtering in 1 step

In [5]:
filtered_pores_3 = pore_filtering.complete_filtering(volume)

100%|██████████| 137206/137206 [00:02<00:00, 64633.35it/s]


In [7]:
filtered_final = ((filtered_pores_2 > 0) * 255).astype(np.uint8)

#compare filtered_pores_2 and filtered_pores_3 to see if they are the same
if np.array_equal(filtered_final, filtered_pores_3):
    print("The two filtered volumes are the same.")
else:
    print("The two filtered volumes are different.")

The two filtered volumes are the same.


# Saving

In [ ]:
filtered_final = ((filtered_pores_2 > 0) * 255).astype(np.uint8)

# Save the filtered volume
output_path = file_path.parent / (file_path.stem + '_filtered.tif')

io.save_tif(output_path, filtered_final)